In [ ]:
'''
Initial code was copied from:
https://github.com/jason71995/Keras-GAN-Library
'''

In [40]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose,Activation,BatchNormalization
from keras.optimizers import Adam

from keras.layers import Dense
from keras import initializers
from keras.layers.core import Dropout



noise_dim = 5

def build_generator(input_shape):

    generator = Sequential()
    
    generator.add(Dense(256,
                        input_dim = noise_dim))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(256))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    

    


    
    generator.add(Dense(105, activation='tanh'))
    return generator


def build_discriminator(input_shape):

    discriminator = Sequential()
    discriminator.add(Dense(256,
                            input_dim=105))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    
    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    

        

    
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)

    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    pred_real = K.clip(pred_real,K.epsilon(),1-K.epsilon())
    pred_fake = K.clip(pred_fake,K.epsilon(),1-K.epsilon())

    d_loss = -(K.mean(K.log(pred_real)) + K.mean(K.log(1-pred_fake)))
    g_loss = -K.mean(K.log(pred_fake))

    # get updates of mean and variance in batch normalization layers
    d_updates = discriminator.get_updates_for([d_input])
    g_updates = generator.get_updates_for([noise])

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss],d_updates + d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_updates + g_training_updates)

    return d_train,g_train

In [3]:
# Loading MHAD data for action1, all persons and all repeatations of each person
from utils.data_loader import data_loader
data_object= data_loader(matlab_action_path='../gan/')
myData, mymin, mymax = data_object.actions_normalised([1], twoD_true_or_threeD_false=False)
myData.shape

(117562, 105)

In [41]:
import keras

#from gan_libs.DCGAN import build_generator, build_discriminator, build_functions
# from gan_libs.LSGAN import build_generator, build_discriminator, build_functions
# from gan_libs.SNGAN import build_generator, build_discriminator, build_functions
# from gan_libs.WGAN_GP import build_generator, build_discriminator, build_functions

from utils.common import set_gpu_config, predict_images
from utils.draw_pose import draw_pose
import numpy as np

#set_gpu_config("0",0.5)

epoch = 30 + 1
image_size = (1,1,105)
noise_size = (1,1,5)
batch_size = 16

x_train = myData
np.random.shuffle(x_train)

generator = build_generator(noise_size)
print('gen params:',generator.count_params())
discriminator = build_discriminator(image_size)
print('dis params:',discriminator.count_params())
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

# generator.load_weights("e25_generator.h5".format(e))
# discriminator.load_weights("e25_discriminator.h5".format(e))


number_of_all_data = x_train.shape[0]
number_of_batches = int(number_of_all_data/batch_size)
print('Number of Batches passed in each epoch: ',number_of_batches)

import time
start_time = time.time()
is_first_epoch = True

for e in range(epoch):
    index = 0
    for batch in range(number_of_batches):
        real_images = x_train[index:index+batch_size]
        index =+ batch_size
        real_images.shape = (batch_size,1,1,105)
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        if np.random.randint(low = 0, high = 100) == 1:   
            print ("[{0}/{1}] d_loss: {2:.4}, g_loss: {3:.4}".format(e, epoch, d_loss, g_loss))
            #generating a sample
            image = generator.predict(np.zeros(shape=(1,5)))
            image = np.array(image)
            draw_pose(image.reshape(105),'output',"e{0}_batch{1}".format(e,batch))
    
    if(is_first_epoch):
        elapsed_time = time.time() - start_time 
        print('\n\nTime Taken for single epoch:')
        print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        is_first_epoch = False
    
    if e % 30 == 0 and e > 0:
        generator.save_weights("e{0}_generator.h5".format(e))
        discriminator.save_weights("e{0}_discriminator.h5".format(e))
 
#just monitoring:
##########################################################################################
elapsed_time = time.time() - start_time 
print('\n\n\n\nNumber of parameter for the Generator and discriminator respectively:\n')
print(generator.count_params())
print('')
print(discriminator.count_params())
print('\n\nNumber of Epochs and steps for each epoch:\n')
print('epochs: ',epoch, '   batches: ', number_of_batches)

print('\n\nTime Taken:')
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
##########################################################################################
#NOTE: previously in each epoch 1000 steps were iterated. in each iteration data was permitated and a batch was chosen. 
#now data is shuffled first and there are number_of_data/batch_size steps to be passed in a single epoch. 
# since 200 epochs and 1000 steps each was used, now 25 epochs and ˜7300 steps is used so the balance is kept.

gen params: 96361
dis params: 93185
Number of Batches passed in each epoch:  7347
[0/31] d_loss: 1.266, g_loss: 0.7868
[0/31] d_loss: 0.781, g_loss: 1.445
[0/31] d_loss: 1.009, g_loss: 1.225
[0/31] d_loss: 0.768, g_loss: 1.271
[0/31] d_loss: 0.6791, g_loss: 1.901
[0/31] d_loss: 0.6427, g_loss: 1.894
[0/31] d_loss: 0.5596, g_loss: 1.731
[0/31] d_loss: 0.5683, g_loss: 2.547
[0/31] d_loss: 0.5125, g_loss: 2.172
[0/31] d_loss: 0.4299, g_loss: 2.279
[0/31] d_loss: 0.3365, g_loss: 2.691
[0/31] d_loss: 0.265, g_loss: 4.39
[0/31] d_loss: 0.2027, g_loss: 3.026
[0/31] d_loss: 0.289, g_loss: 3.755
[0/31] d_loss: 0.1671, g_loss: 3.632
[0/31] d_loss: 0.1379, g_loss: 2.86
[0/31] d_loss: 0.0996, g_loss: 5.21
[0/31] d_loss: 0.1347, g_loss: 5.993
[0/31] d_loss: 0.2031, g_loss: 4.473
[0/31] d_loss: 0.3288, g_loss: 4.515
[0/31] d_loss: 0.3631, g_loss: 4.904
[0/31] d_loss: 0.175, g_loss: 4.649
[0/31] d_loss: 0.3015, g_loss: 3.807
[0/31] d_loss: 0.225, g_loss: 5.625
[0/31] d_loss: 0.1687, g_loss: 4.967
[0/

[3/31] d_loss: 0.6367, g_loss: 1.563
[3/31] d_loss: 0.4548, g_loss: 2.521
[3/31] d_loss: 1.593, g_loss: 2.305
[3/31] d_loss: 0.626, g_loss: 3.088
[3/31] d_loss: 0.5941, g_loss: 1.524
[3/31] d_loss: 0.533, g_loss: 2.576
[3/31] d_loss: 1.61, g_loss: 1.288
[3/31] d_loss: 1.265, g_loss: 3.752
[3/31] d_loss: 1.324, g_loss: 2.877
[3/31] d_loss: 0.4893, g_loss: 2.775
[3/31] d_loss: 0.9148, g_loss: 0.4505
[3/31] d_loss: 0.5873, g_loss: 1.993
[3/31] d_loss: 0.6811, g_loss: 3.014
[3/31] d_loss: 0.7681, g_loss: 0.4618
[3/31] d_loss: 1.157, g_loss: 2.726
[3/31] d_loss: 1.044, g_loss: 5.124
[3/31] d_loss: 0.575, g_loss: 3.574
[3/31] d_loss: 0.4928, g_loss: 1.856
[3/31] d_loss: 0.6695, g_loss: 4.209
[3/31] d_loss: 0.6856, g_loss: 3.392
[3/31] d_loss: 0.6366, g_loss: 2.504
[3/31] d_loss: 0.6332, g_loss: 2.081
[3/31] d_loss: 0.6035, g_loss: 2.455
[3/31] d_loss: 2.224, g_loss: 2.186
[3/31] d_loss: 0.4207, g_loss: 2.889
[3/31] d_loss: 0.9063, g_loss: 3.011
[3/31] d_loss: 0.6445, g_loss: 1.354
[3/31] d_l

[6/31] d_loss: 0.8391, g_loss: 4.098
[6/31] d_loss: 0.2987, g_loss: 2.839
[6/31] d_loss: 0.7018, g_loss: 2.994
[6/31] d_loss: 0.4248, g_loss: 2.983
[6/31] d_loss: 0.3056, g_loss: 3.713
[6/31] d_loss: 0.7539, g_loss: 4.557
[6/31] d_loss: 0.492, g_loss: 2.925
[6/31] d_loss: 0.3955, g_loss: 3.936
[6/31] d_loss: 0.5676, g_loss: 3.249
[6/31] d_loss: 0.7601, g_loss: 4.158
[6/31] d_loss: 0.5131, g_loss: 3.366
[6/31] d_loss: 0.3857, g_loss: 3.535
[6/31] d_loss: 0.8718, g_loss: 4.709
[6/31] d_loss: 1.579, g_loss: 4.67
[6/31] d_loss: 0.2331, g_loss: 3.702
[6/31] d_loss: 0.3352, g_loss: 3.623
[6/31] d_loss: 0.5452, g_loss: 1.662
[6/31] d_loss: 0.3576, g_loss: 3.068
[6/31] d_loss: 0.7522, g_loss: 5.178
[6/31] d_loss: 1.182, g_loss: 5.145
[6/31] d_loss: 0.5797, g_loss: 3.268
[6/31] d_loss: 0.2612, g_loss: 3.116
[6/31] d_loss: 1.324, g_loss: 2.285
[7/31] d_loss: 1.045, g_loss: 2.69
[7/31] d_loss: 0.4452, g_loss: 1.762
[7/31] d_loss: 1.258, g_loss: 0.942
[7/31] d_loss: 0.5773, g_loss: 0.9886
[7/31] d

[10/31] d_loss: 0.5857, g_loss: 4.986
[10/31] d_loss: 0.3963, g_loss: 5.809
[10/31] d_loss: 1.001, g_loss: 2.247
[10/31] d_loss: 0.4246, g_loss: 4.04
[10/31] d_loss: 0.6285, g_loss: 4.193
[10/31] d_loss: 0.2498, g_loss: 3.517
[10/31] d_loss: 0.2513, g_loss: 5.727
[10/31] d_loss: 0.2765, g_loss: 2.878
[10/31] d_loss: 0.4819, g_loss: 3.751
[10/31] d_loss: 0.2581, g_loss: 4.99
[10/31] d_loss: 0.2602, g_loss: 3.53
[10/31] d_loss: 0.8649, g_loss: 0.9412
[10/31] d_loss: 0.3565, g_loss: 4.798
[10/31] d_loss: 0.8769, g_loss: 1.36
[10/31] d_loss: 0.3244, g_loss: 4.875
[10/31] d_loss: 0.3494, g_loss: 4.456
[10/31] d_loss: 0.1811, g_loss: 4.682
[10/31] d_loss: 0.1628, g_loss: 4.435
[10/31] d_loss: 0.2411, g_loss: 3.492
[10/31] d_loss: 1.858, g_loss: 3.711
[10/31] d_loss: 0.2944, g_loss: 3.01
[10/31] d_loss: 0.2355, g_loss: 4.899
[10/31] d_loss: 0.3101, g_loss: 4.346
[10/31] d_loss: 0.6543, g_loss: 1.392
[10/31] d_loss: 0.3695, g_loss: 3.289
[10/31] d_loss: 0.4859, g_loss: 3.0
[10/31] d_loss: 0.31

[12/31] d_loss: 1.463, g_loss: 2.5
[12/31] d_loss: 0.7016, g_loss: 4.018
[12/31] d_loss: 0.2486, g_loss: 2.67
[12/31] d_loss: 0.268, g_loss: 7.063
[12/31] d_loss: 0.4782, g_loss: 6.553
[12/31] d_loss: 0.2754, g_loss: 4.209
[13/31] d_loss: 0.1652, g_loss: 6.348
[13/31] d_loss: 0.3353, g_loss: 4.151
[13/31] d_loss: 0.329, g_loss: 6.229
[13/31] d_loss: 0.8238, g_loss: 5.065
[13/31] d_loss: 0.749, g_loss: 4.71
[13/31] d_loss: 0.9115, g_loss: 7.279
[13/31] d_loss: 0.9318, g_loss: 2.521
[13/31] d_loss: 0.2698, g_loss: 4.08
[13/31] d_loss: 0.3072, g_loss: 6.165
[13/31] d_loss: 0.1821, g_loss: 2.62
[13/31] d_loss: 0.1336, g_loss: 4.973
[13/31] d_loss: 0.3139, g_loss: 6.234
[13/31] d_loss: 0.3796, g_loss: 4.175
[13/31] d_loss: 1.351, g_loss: 1.938
[13/31] d_loss: 0.1583, g_loss: 4.983
[13/31] d_loss: 1.393, g_loss: 4.261
[13/31] d_loss: 1.28, g_loss: 1.003
[13/31] d_loss: 1.103, g_loss: 1.502
[13/31] d_loss: 0.2223, g_loss: 4.284
[13/31] d_loss: 0.1644, g_loss: 5.613
[13/31] d_loss: 0.4557, g_l

...

[18/31] d_loss: 0.08068, g_loss: 7.857
[18/31] d_loss: 0.1371, g_loss: 4.563
[18/31] d_loss: 0.2441, g_loss: 5.948
[18/31] d_loss: 0.1996, g_loss: 5.482
[18/31] d_loss: 0.3788, g_loss: 6.373
[18/31] d_loss: 0.1205, g_loss: 4.725
[18/31] d_loss: 0.1454, g_loss: 3.858
[18/31] d_loss: 0.5475, g_loss: 2.455
[19/31] d_loss: 0.5426, g_loss: 4.578
[19/31] d_loss: 0.2473, g_loss: 2.23
[19/31] d_loss: 0.5696, g_loss: 5.724
[19/31] d_loss: 0.314, g_loss: 3.739
[19/31] d_loss: 0.3031, g_loss: 5.698
[19/31] d_loss: 0.1925, g_loss: 4.275
[19/31] d_loss: 0.8721, g_loss: 5.159
[19/31] d_loss: 0.2656, g_loss: 4.755
[19/31] d_loss: 0.3967, g_loss: 3.445
[19/31] d_loss: 0.2408, g_loss: 3.569
[19/31] d_loss: 0.3257, g_loss: 3.081
[19/31] d_loss: 0.4429, g_loss: 6.343
[19/31] d_loss: 0.5432, g_loss: 6.038
[19/31] d_loss: 0.3061, g_loss: 4.93
[19/31] d_loss: 0.6894, g_loss: 2.49
[19/31] d_loss: 0.2494, g_loss: 2.822
[19/31] d_loss: 0.2945, g_loss: 3.27
[19/31] d_loss: 0.4494, g_loss: 1.942
[19/31] d_loss: 

[21/31] d_loss: 0.2643, g_loss: 6.095
[21/31] d_loss: 0.1137, g_loss: 3.814
[21/31] d_loss: 0.4809, g_loss: 4.402
[21/31] d_loss: 1.127, g_loss: 6.865
[21/31] d_loss: 0.2808, g_loss: 3.863
[21/31] d_loss: 0.7419, g_loss: 5.432
[21/31] d_loss: 0.3043, g_loss: 8.015
[21/31] d_loss: 0.2828, g_loss: 2.208
[21/31] d_loss: 0.5347, g_loss: 5.545
[21/31] d_loss: 0.4479, g_loss: 4.51
[21/31] d_loss: 0.1726, g_loss: 4.709
[21/31] d_loss: 0.6428, g_loss: 1.281
[21/31] d_loss: 1.224, g_loss: 1.645
[21/31] d_loss: 0.3845, g_loss: 4.81
[21/31] d_loss: 0.2479, g_loss: 5.557
[21/31] d_loss: 0.564, g_loss: 5.316
[21/31] d_loss: 0.5063, g_loss: 4.95
[21/31] d_loss: 0.2086, g_loss: 4.152
[21/31] d_loss: 0.2337, g_loss: 4.189
[21/31] d_loss: 0.2502, g_loss: 2.509
[21/31] d_loss: 0.8067, g_loss: 4.123
[21/31] d_loss: 0.199, g_loss: 4.317
[21/31] d_loss: 0.2748, g_loss: 3.795
[21/31] d_loss: 0.2037, g_loss: 7.208
[21/31] d_loss: 0.4474, g_loss: 8.732
[21/31] d_loss: 0.5785, g_loss: 2.682
[21/31] d_loss: 0.2

[24/31] d_loss: 0.433, g_loss: 4.498
[24/31] d_loss: 0.1775, g_loss: 6.832
[24/31] d_loss: 0.3234, g_loss: 4.895
[24/31] d_loss: 0.4095, g_loss: 4.414
[24/31] d_loss: 0.5755, g_loss: 4.152
[24/31] d_loss: 1.379, g_loss: 5.112
[24/31] d_loss: 0.1188, g_loss: 5.16
[24/31] d_loss: 0.5762, g_loss: 3.655
[24/31] d_loss: 0.4517, g_loss: 5.067
[24/31] d_loss: 0.1681, g_loss: 5.592
[24/31] d_loss: 0.8123, g_loss: 5.273
[24/31] d_loss: 0.4716, g_loss: 5.144
[24/31] d_loss: 0.5768, g_loss: 4.934
[24/31] d_loss: 0.5569, g_loss: 4.082
[24/31] d_loss: 0.3966, g_loss: 1.695
[24/31] d_loss: 0.5818, g_loss: 5.791
[24/31] d_loss: 0.2404, g_loss: 4.449
[24/31] d_loss: 0.4543, g_loss: 5.35
[24/31] d_loss: 0.4039, g_loss: 3.879
[24/31] d_loss: 0.3743, g_loss: 5.027
[24/31] d_loss: 0.5507, g_loss: 4.048
[24/31] d_loss: 0.144, g_loss: 5.912
[24/31] d_loss: 0.2294, g_loss: 5.586
[24/31] d_loss: 0.2918, g_loss: 3.711
[24/31] d_loss: 0.3141, g_loss: 4.396
[24/31] d_loss: 0.3378, g_loss: 4.108
[24/31] d_loss: 0

[27/31] d_loss: 0.5259, g_loss: 5.949
[27/31] d_loss: 0.6141, g_loss: 6.308
[27/31] d_loss: 0.4769, g_loss: 5.606
[27/31] d_loss: 0.1326, g_loss: 10.47
[27/31] d_loss: 0.4241, g_loss: 4.573
[27/31] d_loss: 0.3767, g_loss: 5.657
[27/31] d_loss: 0.475, g_loss: 5.01
[27/31] d_loss: 0.3514, g_loss: 7.19
[27/31] d_loss: 0.2606, g_loss: 4.059
[27/31] d_loss: 0.3069, g_loss: 7.469
[27/31] d_loss: 0.2833, g_loss: 5.63
[27/31] d_loss: 0.3842, g_loss: 3.906
[27/31] d_loss: 0.3555, g_loss: 3.085
[27/31] d_loss: 0.7962, g_loss: 4.95
[27/31] d_loss: 0.832, g_loss: 4.421
[27/31] d_loss: 0.4942, g_loss: 6.532
[27/31] d_loss: 0.3053, g_loss: 4.363
[27/31] d_loss: 0.1904, g_loss: 4.541
[27/31] d_loss: 0.2927, g_loss: 3.775
[27/31] d_loss: 0.3895, g_loss: 12.13
[27/31] d_loss: 0.6479, g_loss: 4.369
[27/31] d_loss: 0.4962, g_loss: 5.838
[27/31] d_loss: 0.2779, g_loss: 5.827
[27/31] d_loss: 0.5687, g_loss: 4.153
[27/31] d_loss: 0.6189, g_loss: 3.861
[27/31] d_loss: 0.3445, g_loss: 4.012
[27/31] d_loss: 0.

[30/31] d_loss: 0.295, g_loss: 6.837
[30/31] d_loss: 0.8094, g_loss: 6.815
[30/31] d_loss: 0.4616, g_loss: 6.031
[30/31] d_loss: 0.3089, g_loss: 4.209
[30/31] d_loss: 0.9379, g_loss: 3.99
[30/31] d_loss: 0.3294, g_loss: 3.285
[30/31] d_loss: 0.2349, g_loss: 7.482
[30/31] d_loss: 0.3337, g_loss: 5.011
[30/31] d_loss: 0.3212, g_loss: 3.855
[30/31] d_loss: 0.6325, g_loss: 5.098
[30/31] d_loss: 0.3683, g_loss: 5.648
[30/31] d_loss: 0.3803, g_loss: 3.012
[30/31] d_loss: 0.2115, g_loss: 6.482
[30/31] d_loss: 0.5858, g_loss: 3.856
[30/31] d_loss: 0.1969, g_loss: 8.208
[30/31] d_loss: 0.3984, g_loss: 5.588
[30/31] d_loss: 0.3715, g_loss: 5.367
[30/31] d_loss: 0.2465, g_loss: 5.418
[30/31] d_loss: 0.1744, g_loss: 4.666
[30/31] d_loss: 0.3739, g_loss: 3.701
[30/31] d_loss: 0.3135, g_loss: 5.346
[30/31] d_loss: 1.236, g_loss: 2.102
[30/31] d_loss: 0.3025, g_loss: 5.501
[30/31] d_loss: 0.1378, g_loss: 7.883
[30/31] d_loss: 1.236, g_loss: 3.534
[30/31] d_loss: 0.5753, g_loss: 5.03
[30/31] d_loss: 0

In [18]:
    #gpu details:
    ################----------------------------
from six.moves import cStringIO as StringIO
import gpustat

gpustats = gpustat.new_query()
fp = StringIO()
gpustats.print_formatted(
     fp=fp, no_color=False, show_user=False,
     show_cmd=False, show_pid=False, show_power=False, show_fan_speed=False)

result = fp.getvalue()
print('\n\n')
print(result)
    ################----------------------------




cbrc-All-Series  Mon May  6 14:40:03 2019
[0] GeForce GTX 1080 Ti | 53'C,   0 % |  6283 / 11172 MB | cbrc(5961M) gdm(16M) gdm(50M) cbrc(114M) cbrc(136M)



In [ ]:
generator.load_weights("e400_generator.h5".format(e))
discriminator.load_weights("e400_discriminator.h5".format(e))

In [ ]:
z = -1;
for i in np.linspace(-3,3,100):
    z+=1
    x= np.array((-i,0,0,i,0)).reshape(1,5)
    image = generator.predict(x)
    image = np.array(image)
    draw_pose(image.reshape(105),'output',"e{0}".format(z))